In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
medium_data = pd.read_csv('/content/medium_data.csv')
medium_data.head(5)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [ ]:
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'\xa0',u' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace('\u200a',' '))

In [ ]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
tokenizer.fit_on_texts(medium_data['title'])
total_words = len(tokenizer.word_index) + 1
print("Total number of words: ", total_words)



Total number of words:  8238


In [ ]:
input_sequences = []
for line in medium_data['title']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# print(input_sequences)
print("Total input sequences: ", len(input_sequences))

Total input sequences:  48461


In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   5, 676,
        68], dtype=int32)

In [ ]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=50, verbose=1)


Epoch 1/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 298s 193ms/step - accuracy: 0.0990 - loss: 7.0254
Epoch 2/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 281s 186ms/step - accuracy: 0.1761 - loss: 5.7433
Epoch 3/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 326s 188ms/step - accuracy: 0.2077 - loss: 4.9044
Epoch 4/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 284s 188ms/step - accuracy: 0.2647 - loss: 4.0702
Epoch 5/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 323s 188ms/step - accuracy: 0.3136 - loss: 3.5358
Epoch 6/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 320s 187ms/step - accuracy: 0.3573 - loss: 3.1692
Epoch 7/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 283s 187ms/step - accuracy: 0.3912 - loss: 2.8978
Epoch 8/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 292s 192ms/step - accuracy: 0.4124 - loss: 2.6992
Epoch 9/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 293s 193ms/step - accuracy: 0.4344 - loss: 2.5880
Epoch 10/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 312s 187ms/step - accuracy: 0.4459 - loss: 2.5182
Epoch 11/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 322s 187ms/step - accuracy: 0.4518 -

In [ ]:
seed_text = "implementation of"
next_words = 2

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted = predicted_probs.argmax(axis=-1)[0]  # Use argmax for prediction
    output_word = tokenizer.index_word[predicted]  # Retrieve the word directly
    seed_text += " " + output_word

print(seed_text)

implementation of rnn lstm
